In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import sys

import matplotlib.pyplot as plt
from matplotlib import colors

from ez_data import *
from pyPulses.utils import sweepMeasureParallelepiped, SweepMeasureParallelepipedConfig

If you wish to use GPIB instruments, make sure NI488.2 is installed.


In [2]:
# Create the Dataset Object
path = r'data/demo'
DS = Dataset(path, storage_backend = 'hdf5')

variable_attrs = dict(
    Vtg = {'long_name': r'$V_{tg}$', 'units': r'V'},
    Vbg = {'long_name': r'$V_{bg}$', 'units': r'V'},
    R   = {'long_name': r'R', 'units': r'$\Omega$'}
)

global_attrs = dict(
    B = {'long_name': r'B', 'units': r'T', 'value': 0.0}
)

DS.add_variable_attrs(variable_attrs)
DS.add_global_attrs(global_attrs)

In [3]:
# Set up the sweep
swept_names = ("Vtg", "Vbg")
measured_names = ("R",)

def post_callback(index, coords, data):
    coord_dict = dict(zip(swept_names, coords))
    data_dict = dict(zip(measured_names, data))
    DS.add_point(index, coord_dict, data_dict)

origin = [5.0, 5.0]
endpoints = np.array([[6.0, 6.0],
                      [6.0,  4.0]])
shape = [101, 11]

def dummy_set(val): pass
def dummy_get(): return np.random.exponential(scale=1e6)

para_config = SweepMeasureParallelepipedConfig(
    setters=(dummy_set, dummy_set),
    getters=(dummy_get,),
    swept_name=swept_names,
    measured_name=measured_names,
    time_per_point=0.01,
    origin=origin,
    endpoints=endpoints,
    shape=shape,
    post_callback=post_callback,
    retain_return=False
)

In [4]:
# Run the sweep
sweepMeasureParallelepiped(para_config)
# write metadata and flush any unsaved data
DS.write_metadata()
DS.flush()

In [3]:
# Load the data as a pandas dataframe

df = DS.load_dataframe()
df

,dim0,dim1,Vtg,Vbg,R
0,0,0,5.0,5.0,2.353518e+06
1,0,1,5.1,4.9,9.963832e+05
2,0,2,5.2,4.8,3.877062e+05
3,0,3,5.3,4.7,1.252825e+06
4,0,4,5.4,4.6,7.626007e+06
...,...,...,...,...,...
106,100,6,6.6,5.4,1.022450e+06
107,100,7,6.7,5.3,3.260752e+05
108,100,8,6.8,5.2,4.543977e+05
109,100,9,6.9,5.1,2.732138e+06


In [4]:
# Load the data as an xarray Dataset

ds = DS.to_xarray()
ds

<xarray.Dataset> Size: 27kB
Dimensions:  (dim0: 101, dim1: 11)
Coordinates:
    Vtg      (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg      (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R        (dim0, dim1) float64 9kB 4.584e+05 2.978e+06 ... 5.928e+05
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [7]:
# Plot the data in the xarray dataset as an example
%matplotlib qt5

ds["R"].plot.pcolormesh(x="Vtg", y ="Vbg", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))
plt.show()

In [7]:
apply_linear_transform(df, np.array([[1, 1],[1, -1]]), ["Vtg", "Vbg"], ("n", "D"))

,dim0,dim1,Vtg,Vbg,R,n,D
0,0,0,5.0,5.0,2.353518e+06,10.0,0.0
1,0,1,5.1,4.9,9.963832e+05,10.0,0.2
2,0,2,5.2,4.8,3.877062e+05,10.0,0.4
3,0,3,5.3,4.7,1.252825e+06,10.0,0.6
4,0,4,5.4,4.6,7.626007e+06,10.0,0.8
...,...,...,...,...,...,...,...
106,100,6,6.6,5.4,1.022450e+06,12.0,1.2
107,100,7,6.7,5.3,3.260752e+05,12.0,1.4
108,100,8,6.8,5.2,4.543977e+05,12.0,1.6
109,100,9,6.9,5.1,2.732138e+06,12.0,1.8


In [9]:
ds = apply_linear_transform(ds, np.array([[1, 1],[1, -1]]), ["Vtg", "Vbg"], ("n", "D"))
ds

<xarray.Dataset> Size: 44kB
Dimensions:  (dim0: 101, dim1: 11)
Coordinates:
    Vtg      (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg      (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
    n        (dim0, dim1) float64 9kB 10.0 10.0 10.0 10.0 ... 12.0 12.0 12.0
    D        (dim0, dim1) float64 9kB 0.0 0.2 0.4 0.6 0.8 ... 1.4 1.6 1.8 2.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R        (dim0, dim1) float64 9kB 4.584e+05 2.978e+06 ... 5.928e+05
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [10]:
# Plot the data in the xarray dataset as an example
%matplotlib qt5

ds["R"].plot.pcolormesh(x="n", y ="D", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))
plt.show()

In [12]:
def nD_from_V(Vtg, Vbg):
    return Vtg + Vbg, Vtg - Vbg

In [13]:
apply_transform(ds, nD_from_V, ["Vtg", "Vbg"], ("n", "D"))

<xarray.Dataset> Size: 44kB
Dimensions:  (dim0: 101, dim1: 11)
Coordinates:
    Vtg      (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg      (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
    n        (dim0, dim1) float64 9kB 10.0 10.0 10.0 10.0 ... 12.0 12.0 12.0
    D        (dim0, dim1) float64 9kB 0.0 0.2 0.4 0.6 0.8 ... 1.4 1.6 1.8 2.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R        (dim0, dim1) float64 9kB 4.584e+05 2.978e+06 ... 5.928e+05
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [14]:
apply_transform(df, nD_from_V, ["Vtg", "Vbg"], ("n", "D"))

,dim0,dim1,Vtg,Vbg,R,n,D
0,0,0,5.0,5.0,2.353518e+06,10.0,0.0
1,0,1,5.1,4.9,9.963832e+05,10.0,0.2
2,0,2,5.2,4.8,3.877062e+05,10.0,0.4
3,0,3,5.3,4.7,1.252825e+06,10.0,0.6
4,0,4,5.4,4.6,7.626007e+06,10.0,0.8
...,...,...,...,...,...,...,...
106,100,6,6.6,5.4,1.022450e+06,12.0,1.2
107,100,7,6.7,5.3,3.260752e+05,12.0,1.4
108,100,8,6.8,5.2,4.543977e+05,12.0,1.6
109,100,9,6.9,5.1,2.732138e+06,12.0,1.8
